# Time-Dependent Schrödinger Equation: Wavepacket Dynamics

## Theoretical Foundation

The **time-dependent Schrödinger equation (TDSE)** is the fundamental equation governing the evolution of quantum mechanical systems. For a single particle in one dimension, it reads:

$$i\hbar \frac{\partial \Psi(x,t)}{\partial t} = \hat{H}\Psi(x,t)$$

where the Hamiltonian operator is:

$$\hat{H} = -\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2} + V(x)$$

### Gaussian Wavepacket

We consider a Gaussian wavepacket as our initial condition:

$$\Psi(x,0) = \left(\frac{1}{2\pi\sigma^2}\right)^{1/4} \exp\left(-\frac{(x-x_0)^2}{4\sigma^2}\right) \exp\left(ik_0 x\right)$$

where:
- $x_0$ is the initial center position
- $\sigma$ is the initial width (uncertainty in position)
- $k_0$ is the initial wave vector (related to momentum via $p_0 = \hbar k_0$)

### Numerical Method: Split-Operator Technique

We employ the **split-operator method** (also known as the split-step Fourier method) for time evolution. The formal solution is:

$$\Psi(x, t+\Delta t) = e^{-i\hat{H}\Delta t/\hbar}\Psi(x,t)$$

Using the Trotter-Suzuki decomposition:

$$e^{-i\hat{H}\Delta t/\hbar} \approx e^{-iV\Delta t/(2\hbar)} \cdot e^{-i\hat{T}\Delta t/\hbar} \cdot e^{-iV\Delta t/(2\hbar)} + \mathcal{O}(\Delta t^3)$$

where $\hat{T} = -\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2}$ is the kinetic energy operator.

The kinetic propagator is diagonal in momentum space:

$$\tilde{\Psi}(k, t+\Delta t) = e^{-i\hbar k^2 \Delta t/(2m)} \tilde{\Psi}(k, t)$$

This makes the FFT-based split-operator method highly efficient.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Physical constants (atomic units: hbar = m = 1)
hbar = 1.0
m = 1.0

# Spatial grid parameters
N = 1024  # Number of grid points
L = 100.0  # Domain size [-L/2, L/2]
dx = L / N
x = np.linspace(-L/2, L/2, N, endpoint=False)

# Momentum grid (for FFT)
dk = 2 * np.pi / L
k = np.fft.fftfreq(N, d=dx) * 2 * np.pi

# Time parameters
dt = 0.05
n_steps = 400
frames_to_save = 100
save_interval = n_steps // frames_to_save

print(f"Grid: N={N}, dx={dx:.4f}, L={L}")
print(f"Time: dt={dt}, total_time={dt*n_steps}, frames={frames_to_save}")

In [ ]:
def gaussian_wavepacket(x, x0, sigma, k0):
    """
    Initialize a Gaussian wavepacket.
    
    Parameters:
    -----------
    x : ndarray
        Spatial grid
    x0 : float
        Initial center position
    sigma : float
        Initial width (position uncertainty)
    k0 : float
        Initial wave vector (momentum/hbar)
    
    Returns:
    --------
    psi : ndarray
        Normalized wavefunction
    """
    norm = (1.0 / (2.0 * np.pi * sigma**2))**0.25
    psi = norm * np.exp(-(x - x0)**2 / (4.0 * sigma**2)) * np.exp(1j * k0 * x)
    return psi


def potential_barrier(x, V0, width, center):
    """
    Rectangular potential barrier.
    
    Parameters:
    -----------
    x : ndarray
        Spatial grid
    V0 : float
        Barrier height
    width : float
        Barrier width
    center : float
        Barrier center position
    
    Returns:
    --------
    V : ndarray
        Potential array
    """
    V = np.zeros_like(x)
    mask = np.abs(x - center) < width / 2
    V[mask] = V0
    return V

In [ ]:
def split_operator_step(psi, V, k, dt, hbar, m):
    """
    Perform one time step using the split-operator method.
    
    The evolution operator is split as:
    exp(-iH*dt/hbar) ≈ exp(-iV*dt/2hbar) * exp(-iT*dt/hbar) * exp(-iV*dt/2hbar)
    
    Parameters:
    -----------
    psi : ndarray
        Current wavefunction
    V : ndarray
        Potential energy array
    k : ndarray
        Momentum grid
    dt : float
        Time step
    hbar, m : float
        Physical constants
    
    Returns:
    --------
    psi_new : ndarray
        Wavefunction after time step
    """
    # Half step in position space (potential)
    psi = psi * np.exp(-1j * V * dt / (2.0 * hbar))
    
    # Full step in momentum space (kinetic)
    psi_k = np.fft.fft(psi)
    psi_k = psi_k * np.exp(-1j * hbar * k**2 * dt / (2.0 * m))
    psi = np.fft.ifft(psi_k)
    
    # Half step in position space (potential)
    psi = psi * np.exp(-1j * V * dt / (2.0 * hbar))
    
    return psi


def compute_expectation_values(psi, x, k, dx, hbar):
    """
    Compute expectation values of position and momentum.
    
    Returns:
    --------
    x_exp, p_exp : float
        Expectation values <x> and <p>
    """
    prob = np.abs(psi)**2
    x_exp = np.sum(x * prob) * dx
    
    psi_k = np.fft.fft(psi) * dx / np.sqrt(2 * np.pi)
    prob_k = np.abs(psi_k)**2
    p_exp = np.sum(hbar * k * prob_k) * (k[1] - k[0]) / (2 * np.pi)
    
    return x_exp, p_exp

## Simulation Setup

We simulate a Gaussian wavepacket incident on a rectangular potential barrier, demonstrating **quantum tunneling**. The classical kinetic energy of the particle is:

$$E_k = \frac{\hbar^2 k_0^2}{2m}$$

We set the barrier height $V_0 > E_k$ so that classically the particle cannot penetrate the barrier, but quantum mechanically there is a finite tunneling probability.

In [ ]:
# Wavepacket parameters
x0 = -20.0  # Initial position (left of barrier)
sigma = 3.0  # Initial width
k0 = 1.5  # Initial wave vector (moving right)

# Barrier parameters
V0 = 1.5  # Barrier height
barrier_width = 2.0
barrier_center = 0.0

# Calculate classical kinetic energy
E_kinetic = hbar**2 * k0**2 / (2 * m)
print(f"Initial kinetic energy: E_k = {E_kinetic:.3f}")
print(f"Barrier height: V_0 = {V0:.3f}")
print(f"Energy ratio E_k/V_0 = {E_kinetic/V0:.3f}")
print(f"Classical prediction: {'Transmission' if E_kinetic > V0 else 'Reflection'}")

# Initialize wavefunction and potential
psi = gaussian_wavepacket(x, x0, sigma, k0)
V = potential_barrier(x, V0, barrier_width, barrier_center)

# Verify normalization
norm = np.sum(np.abs(psi)**2) * dx
print(f"Initial normalization: {norm:.6f}")

In [ ]:
# Run simulation and store frames
psi_frames = []
time_values = []
x_expectation = []
normalization = []

psi_current = psi.copy()

for step in range(n_steps):
    if step % save_interval == 0:
        psi_frames.append(psi_current.copy())
        time_values.append(step * dt)
        
        # Track observables
        x_exp, _ = compute_expectation_values(psi_current, x, k, dx, hbar)
        x_expectation.append(x_exp)
        normalization.append(np.sum(np.abs(psi_current)**2) * dx)
    
    psi_current = split_operator_step(psi_current, V, k, dt, hbar, m)

# Add final frame
psi_frames.append(psi_current.copy())
time_values.append(n_steps * dt)
x_exp, _ = compute_expectation_values(psi_current, x, k, dx, hbar)
x_expectation.append(x_exp)
normalization.append(np.sum(np.abs(psi_current)**2) * dx)

print(f"Simulation complete: {len(psi_frames)} frames stored")
print(f"Final normalization: {normalization[-1]:.6f} (should be ~1.0)")

In [ ]:
# Calculate transmission and reflection coefficients
# Transmitted: x > barrier_center + barrier_width/2
# Reflected: x < barrier_center - barrier_width/2

final_psi = psi_frames[-1]
prob_density = np.abs(final_psi)**2

transmitted_mask = x > (barrier_center + barrier_width/2 + 5.0)  # Well past barrier
reflected_mask = x < (barrier_center - barrier_width/2 - 5.0)  # Well before barrier

T = np.sum(prob_density[transmitted_mask]) * dx  # Transmission coefficient
R = np.sum(prob_density[reflected_mask]) * dx  # Reflection coefficient

print(f"\nQuantum Tunneling Results:")
print(f"Transmission coefficient T = {T:.4f}")
print(f"Reflection coefficient R = {R:.4f}")
print(f"T + R = {T + R:.4f} (should approach 1.0)")

In [ ]:
# Create animation figure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Time-Dependent Schrödinger Equation: Quantum Tunneling', fontsize=14, fontweight='bold')

# Main wavefunction plot
ax1 = axes[0, 0]
ax1.set_xlim(-L/2, L/2)
ax1.set_ylim(-0.05, 0.35)
ax1.set_xlabel('Position x', fontsize=11)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title('Wavefunction Evolution', fontsize=12)

# Plot potential (scaled for visibility)
V_scaled = V / (V0 * 5)  # Scale to fit nicely
ax1.fill_between(x, 0, V_scaled, alpha=0.3, color='gray', label='Potential barrier')
ax1.axhline(y=E_kinetic/(V0*5), color='red', linestyle='--', alpha=0.5, label=f'$E_k$ = {E_kinetic:.2f}')

line_prob, = ax1.plot([], [], 'b-', lw=2, label='$|\\Psi|^2$')
line_real, = ax1.plot([], [], 'g-', lw=1, alpha=0.7, label='Re($\\Psi$)')
line_imag, = ax1.plot([], [], 'r-', lw=1, alpha=0.7, label='Im($\\Psi$)')
ax1.legend(loc='upper right', fontsize=9)

# Expectation value plot
ax2 = axes[0, 1]
ax2.set_xlim(0, time_values[-1])
ax2.set_ylim(-25, 35)
ax2.set_xlabel('Time t', fontsize=11)
ax2.set_ylabel('$\\langle x \\rangle$', fontsize=11)
ax2.set_title('Position Expectation Value', fontsize=12)
ax2.axhline(y=barrier_center, color='gray', linestyle='--', alpha=0.5)
ax2.axvspan(0, time_values[-1], ymin=0.48, ymax=0.52, alpha=0.2, color='gray')
line_xexp, = ax2.plot([], [], 'b-', lw=2)
point_xexp, = ax2.plot([], [], 'ro', ms=8)

# Momentum space plot
ax3 = axes[1, 0]
ax3.set_xlim(-5, 5)
ax3.set_ylim(0, 3)
ax3.set_xlabel('Wave vector k', fontsize=11)
ax3.set_ylabel('$|\\tilde{\\Psi}(k)|^2$', fontsize=11)
ax3.set_title('Momentum Space Distribution', fontsize=12)
ax3.axvline(x=k0, color='red', linestyle='--', alpha=0.5, label=f'$k_0$ = {k0}')
ax3.axvline(x=-k0, color='orange', linestyle='--', alpha=0.5, label=f'$-k_0$ (reflected)')
line_mom, = ax3.plot([], [], 'b-', lw=2)
ax3.legend(loc='upper right', fontsize=9)

# Info text panel
ax4 = axes[1, 1]
ax4.axis('off')
info_text = ax4.text(0.1, 0.5, '', fontsize=11, family='monospace',
                     verticalalignment='center', transform=ax4.transAxes)

time_text = ax1.text(0.02, 0.95, '', transform=ax1.transAxes, fontsize=11,
                     verticalalignment='top', fontweight='bold')

plt.tight_layout()

def init():
    line_prob.set_data([], [])
    line_real.set_data([], [])
    line_imag.set_data([], [])
    line_xexp.set_data([], [])
    point_xexp.set_data([], [])
    line_mom.set_data([], [])
    time_text.set_text('')
    info_text.set_text('')
    return line_prob, line_real, line_imag, line_xexp, point_xexp, line_mom, time_text, info_text

def animate(frame):
    psi_frame = psi_frames[frame]
    t = time_values[frame]
    
    # Probability density
    prob = np.abs(psi_frame)**2
    line_prob.set_data(x, prob)
    line_real.set_data(x, np.real(psi_frame) * 0.5 + 0.15)  # Offset for visibility
    line_imag.set_data(x, np.imag(psi_frame) * 0.5 + 0.15)
    
    # Expectation value trajectory
    line_xexp.set_data(time_values[:frame+1], x_expectation[:frame+1])
    point_xexp.set_data([t], [x_expectation[frame]])
    
    # Momentum space
    psi_k = np.fft.fft(psi_frame) * dx / np.sqrt(2 * np.pi)
    psi_k = np.fft.fftshift(psi_k)
    k_shifted = np.fft.fftshift(k)
    prob_k = np.abs(psi_k)**2
    line_mom.set_data(k_shifted, prob_k)
    
    time_text.set_text(f't = {t:.2f}')
    
    # Calculate current transmission/reflection
    trans_mask = x > (barrier_center + barrier_width/2)
    refl_mask = x < (barrier_center - barrier_width/2)
    T_current = np.sum(prob[trans_mask]) * dx
    R_current = np.sum(prob[refl_mask]) * dx
    
    info_str = f"""Time-Dependent Schrödinger Equation
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Parameters:
  • Initial momentum: k₀ = {k0:.2f}
  • Kinetic energy: Eₖ = {E_kinetic:.3f}
  • Barrier height: V₀ = {V0:.3f}
  • Barrier width: w = {barrier_width:.1f}

Current State (t = {t:.2f}):
  • ⟨x⟩ = {x_expectation[frame]:.2f}
  • Transmitted: {T_current*100:.1f}%
  • Reflected: {R_current*100:.1f}%
  • Norm: {normalization[frame]:.6f}

Classical Prediction: REFLECTION
Quantum Reality: PARTIAL TUNNELING
"""
    info_text.set_text(info_str)
    
    return line_prob, line_real, line_imag, line_xexp, point_xexp, line_mom, time_text, info_text

anim = FuncAnimation(fig, animate, init_func=init, frames=len(psi_frames),
                     interval=50, blit=True)

plt.close()  # Prevent duplicate display
HTML(anim.to_jshtml())

In [ ]:
# Generate static summary figure for plot.png
fig_static, axes_static = plt.subplots(2, 2, figsize=(14, 10))
fig_static.suptitle('Time-Dependent Schrödinger Equation: Quantum Tunneling Summary', 
                    fontsize=14, fontweight='bold')

# Panel 1: Initial vs Final wavefunction
ax1 = axes_static[0, 0]
ax1.fill_between(x, 0, V/V0 * 0.3, alpha=0.3, color='gray', label='Barrier (scaled)')
ax1.plot(x, np.abs(psi_frames[0])**2, 'b-', lw=2, label='Initial $|\\Psi|^2$')
ax1.plot(x, np.abs(psi_frames[-1])**2, 'r-', lw=2, label='Final $|\\Psi|^2$')
ax1.set_xlim(-L/2, L/2)
ax1.set_xlabel('Position x', fontsize=11)
ax1.set_ylabel('Probability Density', fontsize=11)
ax1.set_title('Wavepacket: Initial vs Final State', fontsize=12)
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Panel 2: Time evolution snapshots
ax2 = axes_static[0, 1]
snapshot_indices = [0, len(psi_frames)//4, len(psi_frames)//2, 3*len(psi_frames)//4, -1]
colors = plt.cm.viridis(np.linspace(0, 1, len(snapshot_indices)))

ax2.fill_between(x, 0, V/V0 * 0.25, alpha=0.2, color='gray')
for idx, (snap_idx, color) in enumerate(zip(snapshot_indices, colors)):
    t_snap = time_values[snap_idx]
    ax2.plot(x, np.abs(psi_frames[snap_idx])**2 + idx*0.02, 
             color=color, lw=1.5, label=f't = {t_snap:.1f}')

ax2.set_xlim(-40, 40)
ax2.set_xlabel('Position x', fontsize=11)
ax2.set_ylabel('Probability Density (offset)', fontsize=11)
ax2.set_title('Time Evolution Snapshots', fontsize=12)
ax2.legend(fontsize=9, loc='upper right')
ax2.grid(True, alpha=0.3)

# Panel 3: Expectation value trajectory
ax3 = axes_static[1, 0]
ax3.plot(time_values, x_expectation, 'b-', lw=2)
ax3.axhline(y=barrier_center, color='red', linestyle='--', alpha=0.7, label='Barrier center')
ax3.axhspan(barrier_center - barrier_width/2, barrier_center + barrier_width/2, 
            alpha=0.2, color='gray', label='Barrier region')
ax3.set_xlabel('Time t', fontsize=11)
ax3.set_ylabel('$\\langle x \\rangle$', fontsize=11)
ax3.set_title('Position Expectation Value vs Time', fontsize=12)
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Panel 4: Initial vs Final momentum distribution
ax4 = axes_static[1, 1]
k_shifted = np.fft.fftshift(k)

psi_k_init = np.fft.fft(psi_frames[0]) * dx / np.sqrt(2 * np.pi)
psi_k_init = np.fft.fftshift(psi_k_init)

psi_k_final = np.fft.fft(psi_frames[-1]) * dx / np.sqrt(2 * np.pi)
psi_k_final = np.fft.fftshift(psi_k_final)

ax4.plot(k_shifted, np.abs(psi_k_init)**2, 'b-', lw=2, label='Initial')
ax4.plot(k_shifted, np.abs(psi_k_final)**2, 'r-', lw=2, label='Final')
ax4.axvline(x=k0, color='green', linestyle='--', alpha=0.7, label=f'$k_0$ = {k0}')
ax4.axvline(x=-k0, color='orange', linestyle='--', alpha=0.7, label=f'$-k_0$ (reflected)')
ax4.set_xlim(-5, 5)
ax4.set_xlabel('Wave vector k', fontsize=11)
ax4.set_ylabel('$|\\tilde{\\Psi}(k)|^2$', fontsize=11)
ax4.set_title('Momentum Space Distribution', fontsize=12)
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)

plt.tight_layout()

# Save the figure
fig_static.savefig('plot.png', dpi=150, bbox_inches='tight', facecolor='white')
print("Saved static summary to plot.png")

plt.show()

## Results and Discussion

### Quantum Tunneling Phenomenon

The simulation demonstrates one of the most striking features of quantum mechanics: **quantum tunneling**. Despite the particle's kinetic energy being less than the barrier height ($E_k < V_0$), a portion of the wavepacket transmits through the barrier.

### Key Observations

1. **Wavepacket Splitting**: Upon encountering the barrier, the wavepacket splits into transmitted and reflected components.

2. **Momentum Reversal**: The reflected component shows a peak at $-k_0$ in momentum space, indicating reversed momentum.

3. **Conservation of Probability**: The normalization $\int|\Psi|^2 dx = 1$ is preserved throughout the simulation, verifying unitarity.

4. **Phase Accumulation**: The transmitted wave accumulates a phase shift from tunneling through the classically forbidden region.

### Analytical Comparison

For a rectangular barrier, the transmission coefficient can be approximated (for $E < V_0$) by:

$$T \approx \frac{16 E(V_0 - E)}{V_0^2} e^{-2\kappa w}$$

where $\kappa = \sqrt{2m(V_0 - E)}/\hbar$ and $w$ is the barrier width.

### Numerical Method Validation

The split-operator method is:
- **Symplectic**: Preserves the phase-space volume
- **Unitary**: Preserves probability normalization
- **Time-reversible**: Symmetric under time reversal
- **Second-order accurate**: Error scales as $\mathcal{O}(\Delta t^3)$